<a href="https://colab.research.google.com/github/kai-tan/Stocks_Prediction/blob/master/Stock_Prediction_LSTM_3_Months_Horizon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Changing to Percentage Return, using adj close instead. And predicting 3 months Stocks horizon instead of daily. I have added other stock fundamentals ratios as well in the hope that it will improve its prediction power. 

I have used RapidAPI's alpha vantage and morningstar in order to get the data. 

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
from time import time

from tensorflow.python.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers



Using TensorFlow backend.


In [3]:
print(tf.__version__)

2.1.0


# Importing Google Key ratio for the preceding 10 years from 2019 - 2010 

In [0]:
import requests
import pprint
import json

url = "https://morningstar1.p.rapidapi.com/convenient/keyratios"

querystring = {"Mic":"XNAS","Ticker":"GOOGL"}

headers = {
    'x-rapidapi-host': "morningstar1.p.rapidapi.com",
    'x-rapidapi-key': "### REPLACE WITH YOUR OWN KEY ###",   
    'accept': "string"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

result_dict = json.loads(response.text)


In [0]:
startDate = []
endDate = []

bookValuePerShare = [] 
earningsPerShare = []
freeCashFlowPerShare = []
grossMarginPercent = []
operatingMarginPercent = []

rAndD = []
assetTurnoverOnAverage = [] 
financialLeverageOnAverage = []
netMarginPercent = []
returnOnAssetsPercent = []
returnOnEquityPercent = []
returnOnInvestedCapitalPercent = []
taxRatePercent = []
threeYearAverage = []
freeCashFlowOrNetIncome = []
freeCashFlowOrSalesPercent = []
operatingCashFlowGrowthPercentYoy = []
currentRatio = []
debtOrEquity = []
financialLeverage = []
quickRatio = []
assetTurnover = []
daysSalesOutstanding = []
fixedAssetsTurnover = []
payablesPeriod = []
receivablesTurnover = []

In [9]:
result_dict

{'message': 'Oops, an error occurred. Please contact support@rapidapi.com'}

In [8]:
for i in result_dict['result']: 

  startDate.append(i['startDate'])
  endDate.append(i['endDate'])
  for j, k in i['keyRatioFinancialsSection'].items(): 
    if j == 'bookValuePerShare':
      bookValuePerShare.append(k)
    elif j == 'earningsPerShare':
      earningsPerShare.append(k)
    elif j == 'freeCashFlowPerShare':
      freeCashFlowPerShare.append(k)
    elif j == 'grossMarginPercent': 
      grossMarginPercent.append(k)
    elif j == 'operatingMarginPercent':
      operatingMarginPercent.append(k)
  

  rAndD.append(i['keyRatioStatisticsSection']['marginsPercentageOfSalesSection']['rAndD'])
  for l, m in i['keyRatioStatisticsSection']['profitabilitySection'].items(): 
    
    if l == 'assetTurnoverOnAverage':
      assetTurnoverOnAverage.append(m)
    elif l == 'financialLeverageOnAverage':
      financialLeverageOnAverage.append(m)
    elif l == 'netMarginPercent':
      netMarginPercent.append(m)
    elif l == 'returnOnAssetsPercent': 
      returnOnAssetsPercent.append(m)
    elif l == 'returnOnEquityPercent':
      returnOnEquityPercent.append(m)
    elif l == 'returnOnInvestedCapitalPercent':
      returnOnInvestedCapitalPercent.append(m)
    elif l == 'taxRatePercent':
      taxRatePercent.append(m)
  
  threeYearAverage.append(i['keyRatioStatisticsSection']['growthSection']['revenueGrowthSection']['threeYearAverage'])

  for n, o in i['keyRatioStatisticsSection']['cashflowRatiosSection'].items(): 
    if n == 'freeCashFlowOrNetIncome':
      freeCashFlowOrNetIncome.append(o)
    elif n == 'freeCashFlowOrSalesPercent':
      freeCashFlowOrSalesPercent.append(o)
    elif n == 'operatingCashFlowGrowthPercentYoy':
      operatingCashFlowGrowthPercentYoy.append(o)

  for p, q in i['keyRatioStatisticsSection']['financialHealthLiquiditySection'].items(): 
    if p == 'currentRatio':
      currentRatio.append(q)
    elif p == 'debtOrEquity':
      debtOrEquity.append(q)
    elif p == 'financialLeverage':
      financialLeverage.append(q)
    elif p == 'quickRatio':
      quickRatio.append(q)
  
  for s, t in i['keyRatioStatisticsSection']['efficiencyRatiosSection'].items(): 
    if s == 'assetTurnover':
      assetTurnover.append(t)
    elif s == 'daysSalesOutstanding':
      daysSalesOutstanding.append(t)
    elif s == 'fixedAssetsTurnover':
      fixedAssetsTurnover.append(t)
    elif s == 'payablesPeriod':
      payablesPeriod.append(t)
    elif s == 'receivablesTurnover':
      receivablesTurnover.append(t)

KeyError: ignored

In [0]:
df_ratios = pd.DataFrame({'year': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], 'startDate': startDate, 'endDate': endDate, 'bookValuePerShare': bookValuePerShare, 'earningsPerShare': earningsPerShare, 'freeCashFlowPerShare': freeCashFlowPerShare, 'grossMarginPercent': grossMarginPercent, 'operatingMarginPercent': operatingMarginPercent, 'rAndD': rAndD
, 'assetTurnoverOnAverage': assetTurnoverOnAverage,  
'financialLeverageOnAverage': financialLeverageOnAverage, 
'netMarginPercent': netMarginPercent, 
'returnOnAssetsPercent': returnOnAssetsPercent, 
'returnOnEquityPercent': returnOnEquityPercent, 
'returnOnInvestedCapitalPercent': returnOnInvestedCapitalPercent, 
'taxRatePercent': taxRatePercent, 
'threeYearAverage': threeYearAverage, 
'freeCashFlowOrNetIncome': freeCashFlowOrNetIncome, 
'freeCashFlowOrSalesPercent': freeCashFlowOrSalesPercent, 
'operatingCashFlowGrowthPercentYoy': operatingCashFlowGrowthPercentYoy, 
'currentRatio': currentRatio, 
'debtOrEquity': debtOrEquity, 
'financialLeverage': financialLeverage, 
'quickRatio': quickRatio, 
'assetTurnover': assetTurnover, 
'daysSalesOutstanding': daysSalesOutstanding, 
'fixedAssetsTurnover': fixedAssetsTurnover, 
'payablesPeriod': payablesPeriod, 
'receivablesTurnover': receivablesTurnover})

In [0]:
df_ratios

In [0]:
df_ratio_num = df_ratios.drop(['endDate', 'startDate', 'year'], axis=1)

In [0]:
df_ratio_num.diff(axis=0).mean()

In [0]:
url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"outputsize":"full","datatype":"json","function":"TIME_SERIES_DAILY_ADJUSTED","symbol":"GOOGL"}

headers = {
    'x-rapidapi-host': "alpha-vantage.p.rapidapi.com",
    'x-rapidapi-key': "### REPLACE WITH YOUR OWN KEY ###"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

result_dict = json.loads(response.text)
result_dict

In [0]:
date_data = []

high = []
low = []
adjustedClose = []
volume = []

In [0]:
for i, j in result_dict['Time Series (Daily)'].items():

  date_data.append(i)

  for k, l in j.items():
    
    if k == '2. high': 
      high.append(l)
    elif k == '3. low':
      low.append(l)
    elif k == '5. adjusted close': 
      adjustedClose.append(l)
    elif k == '6. volume': 
      volume.append(l)

In [0]:
df_date = pd.DataFrame({ 'date_data': date_data, 'high': high, 'low': low, 
'adjustedClose': adjustedClose, 'volume': volume })

In [0]:
df_date.dtypes

In [0]:
df_date['date_data'] =  pd.to_datetime(df_date['date_data'], format='%Y-%m-%d')

In [0]:
import datetime

In [0]:
df_date['year'] = df_date['date_data'].dt.year

In [0]:
df_date = df_date.merge(df_ratios,how='outer',  left_on='year', right_on='year')

In [0]:
df_date = df_date.drop(['year', 'startDate', 'endDate'],axis=1)

In [0]:
df_date

In [0]:
df_date = df_date.interpolate(limit_direction='both')

In [0]:
df_date

In [0]:
data = df_date.rename(columns={'date_data': 'Date', 'adjustedClose': 'Adj Close'})


In [0]:
data

In [0]:
def adjclosefromdate(df, date): 
    return float(df[df['Date'] == date]['Adj Close'])

In [0]:
adjCloseBeginning = adjclosefromdate(data, '2004-08-19')

In [0]:
data['Adj Close'] = data['Adj Close'].astype(np.float)

In [0]:
data['% Change In Adj Close'] = (100 * (data['Adj Close'] - adjCloseBeginning))/adjCloseBeginning

In [0]:
data_training = data[data['Date']<'2019-01-01'].copy()
data_training

In [0]:
data_test = data[data['Date']>='2019-01-01'].copy()
data_test

In [0]:
training_data_y = data_training['% Change In Adj Close']

In [0]:
training_data = data_training.drop(['Date','high', 'low', 'volume', '% Change In Adj Close'], axis=1)
training_data.columns

In [0]:
training_data.shape

In [0]:
training_data_y

In [0]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

In [0]:
scaler2 = MinMaxScaler()
training_data_y = scaler.fit_transform(np.array(training_data_y).reshape(-1, 1))
training_data_y

In [0]:
training_data_y = np.squeeze(training_data_y)

In [0]:
training_data_y.shape

In [0]:
X_train = []
y_train = []

In [0]:
training_data.shape

In [0]:
# for i in range(60, training_data.shape[0]):
#     X_train.append(training_data[i-60:i])
#     y_train.append(training_data[i, 0])

for i in range(180, training_data.shape[0]):
  if i + 90 >= training_data.shape[0]: 
    break
  else:
    X_train.append(training_data[i-180: i])
    y_train.append(training_data_y[i+90])

In [0]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [0]:
X_train.shape, y_train.shape

In [0]:
np.any(np.isnan(X_train))

In [0]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Dropout 

In [0]:
regressor = Sequential()

regressor.add(LSTM(units = 180, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 27), kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 180, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
regressor.add(Dropout(0.3))

# regressor.add(LSTM(units = 180, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
# regressor.add(Dropout(0.4))

regressor.add(LSTM(units = 180, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
regressor.add(Dropout(0.5))

regressor.add(Dense(units = 1))

In [0]:
regressor.summary()

In [0]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Adam', clipnorm=1, clipvalue=0.5)

In [0]:
NAME = 'Basic_Stock_Prediction_{}'.format(time())

In [0]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [0]:
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [0]:
modelcheckpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)


In [0]:
regressor.compile(optimizer=adam, loss='mean_squared_error')


In [0]:
history = regressor.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.15, callbacks=[tensorboard, earlystopping, modelcheckpoint])


In [0]:
plot_df = pd.DataFrame(history.history)
plot_df.plot()

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs

# Evaluating test set and plot on the test set 

In [0]:
data_test.head()

In [0]:
data_test

In [0]:
past_180_days = data_training.head(180)

In [0]:
past_180_days.head()

In [0]:
df = data_test.append(past_180_days, ignore_index=True)

In [0]:
inputs_y = df['% Change In Adj Close']

In [0]:
inputs_y = np.array(inputs_y).reshape(-1, 1)

In [0]:
inputs_y.shape

In [0]:
df = df.drop(['Date', 'high', 'low', 'volume', '% Change In Adj Close'], axis = 1)
df.head()

In [0]:
df.shape

In [0]:
inputs = scaler.transform(df)
inputs

In [0]:
inputs_y = scaler1.transform(inputs_y)
inputs_y

In [0]:
X_test = []
y_test = [] 

# for i in range(60, inputs.shape[0]):
#   X_test.append(inputs[i-60:i])
#   y_test.append(inputs[i, 0])

for i in range(180, inputs.shape[0]):
  if i + 90 >= inputs.shape[0]: 
    break
  else:
    X_test.append(inputs[i-180: i])
    y_test.append(np.squeeze(inputs_y)[i+90])

In [0]:
X_test, y_test = np.array(X_test), np.array(y_test)

In [0]:
X_test.shape, y_test.shape

In [0]:
results = regressor.evaluate(X_test, y_test, batch_size=128)
print('test loss, test acc:', results)

In [0]:
y_pred = regressor.predict(X_test)

In [0]:
y_pred = scaler1.inverse_transform(np.array(y_pred).reshape(-1, 1))

In [0]:
y_test = scaler1.inverse_transform(np.array(y_test).reshape(-1, 1))

In [0]:
# scaler.scale_

In [0]:
# scale = 1/4.10194301e-04

In [0]:
# y_pred = y_pred*scale

In [0]:
# y_test = y_test*scale

In [0]:
plt.figure(figsize=(14, 5))
plt.plot(y_test, color='red', label='Real % Change in Google Stock Price')
plt.plot(y_pred, color='blue', label='Predicted % Change in Google Stock Price')
plt.title('% Change in Google Stock Price')
plt.xlabel('Time')
plt.ylabel('% Change in Google Stock Price')
plt.legend()
plt.show()

# Miscellaneous: Plot the training sets with label and see how it looks.  

In [0]:
yy_pred = regressor.predict(X_train)

In [0]:
yy_pred = yy_pred*scale

In [0]:
yy_pred.shape

In [0]:
yy_train = y_train*scale

In [0]:
yy_train.shape

In [0]:
yy_train[-1]

In [0]:
additional_array = scaler.inverse_transform(training_data)[180:270, 4]

In [0]:
additional_array

In [0]:
yy_train = np.concatenate((additional_array, yy_train))

In [0]:
yy_train

In [0]:
yyy = pd.Series(yy_train).shift(90)
yyy

In [0]:
yyy.dropna(inplace=True)

In [0]:
yyy.shape

In [0]:
yyy = np.array(yyy)

In [0]:
yyy = yyy[:, np.newaxis]
yyy.shape

In [0]:
z = (yy_pred - yyy)/yyy 
z[-30:]

In [0]:
yyy_train = yy_train[90:, np.newaxis]
yyy_train.shape

In [0]:
yyy_train

In [0]:
zz = (yyy_train - yyy)/yyy 
zz[-30:]

In [0]:
zzz = np.abs(zz - z)
zzz[-30:]

In [0]:
np.mean(zzz)

In [0]:
plt.figure(figsize=(14, 5))
plt.plot(yy_train, color='red', label='Real % Change in Google Stock Price')
plt.plot(yy_pred, color='blue', label='Predicted % Change in Google Stock Price')
plt.title('% Change in Google Stock Price')
plt.xlabel('Time')
plt.ylabel('% Change in Google Stock Price')
plt.legend()
plt.show()
